In [11]:
#!/root/anaconda3/bin/python3
import pandas as pd
from time import sleep
import re
import threading
import random
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import os

url = 'http://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&SID=6AoL5TQ4lX57cBZug4g&search_mode=GeneralSearch'
user_agent_list = [ \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60" \
        "Opera/8.0 (Windows NT 5.1; U; en)"\
        "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0" \
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER" \
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0" \
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)" \
    ]
options = webdriver.ChromeOptions()
UserAgent = 'user-agent=' + random.choice(user_agent_list)
options.add_argument(UserAgent)

data = pd.read_excel('基础数据/外文采集-中英文对照.xlsx')
data = data['英文名称']
# with open('高校英文名称.txt','r',encoding='utf-8') as fin:
#     data = fin.readlines()
# data = [x.strip() for x in data]

with open('外文期刊论文/succeed.txt','r',encoding='utf-8') as f:
    succeed = f.readlines()
succeed = [x.strip() for x in succeed]
for college in data:
    if college in succeed:
        print( college + ' pass')
        continue
#     try:
    index_path = 'D:\\codes\\GitProject\\工作区\\财经院校-学科评估系统建设\\外文期刊论文'
    col_path = index_path+'\\'+college
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': col_path}  #下载路径
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)
    sleep(5)
    input1 = driver.find_element_by_id('container(input1)').find_element_by_tag_name('input')
    input1.clear()
    input1.send_keys(college)
    sleep(1.5)
    search = driver.find_element_by_id('searchCell1').find_element_by_tag_name('button')
    search.click()

    sleep(3)
    results_count = driver.find_element_by_id('hitCount.top').text
    results_count = results_count.replace(',','')
    try:
        counts = int(results_count)
    except:
        print('强制类型转换出错')
        continue
    #导出按钮
    flag = False
    for i in range(200): # 200 * 500 = 100000,只要记录数不超过100000就没问题
        driver.find_element_by_class_name('selectedExportOption').find_element_by_tag_name('button').click()
        from_ = driver.find_element_by_id('markFrom')
        from_.clear()
        from_.send_keys(i*500+1)
        sleep(1)
        to_ =  driver.find_element_by_id('markTo')
        to_.clear()
        if counts < (i+1)*500:
            to_.send_keys(counts)
            flag = True
        else:
            to_.send_keys((i+1)*500)
        Select(driver.find_element_by_id('bib_fields')).select_by_visible_text('全记录') #字段范围
        Select(driver.find_element_by_id('saveOptions')).select_by_visible_text('纯文本') #导出格式
        driver.find_element_by_id('exportButton').click()
        sleep(3)
        driver.find_element_by_xpath('//*[@id="page"]/div[11]/div[1]/button').click()
        if flag:
            break
    print(college + ' 导出成功')
    sleep(5)
    succeed.append(college)
    with open('外文期刊论文/succeed.txt','w',encoding='utf-8') as f:
        for item in succeed:
            f.write(item)
            f.write('\n')
#     except:
#         print('sth is wrong in '+ college)      
        

TimeoutException: Message: timeout
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Windows NT 10.0.17763 x86_64)


In [ ]:
with open('外文期刊论文/succeed.txt','w',encoding='utf-8') as f:
    for item in succeed:
        f.write(item)
        f.write('\n')